In [14]:
import cupy as cp

# Use numpy instead of cupy if you don't have a GPU
def relu(x):
    return cp.maximum(0, x)

def relu_derivative(x):
    return cp.where(x > 0, 1, 0)

def softmax(x):
    e_x = cp.exp(x - cp.max(x, axis=1, keepdims=True))
    return e_x / cp.sum(e_x, axis=1, keepdims=True)

def cross_entropy_loss(predictions, targets):
    return -cp.sum(targets * cp.log(predictions + 1e-9)) / targets.shape[0]

def cross_entropy_loss_with_regularization(predictions, targets, W1, W2, reg_lambda):
    loss = cross_entropy_loss(predictions, targets)
    reg_loss = 0.5 * reg_lambda * (cp.sum(W1**2) + cp.sum(W2**2))
    return loss + reg_loss

def predict(x, W1, b1, W2, b2):
    z1 = cp.dot(x, W1.T) + b1
    a1 = relu(z1)
    z2 = cp.dot(a1, W2.T) + b2
    y_hat = softmax(z2)
    return cp.argmax(y_hat, axis=1)

def initialize_parameters(n_input=784, n_hidden=128, n_output=10):
    cp.random.seed(42)
    W1 = cp.random.randn(n_hidden, n_input) * 0.01
    b1 = cp.zeros(n_hidden)
    W2 = cp.random.randn(n_output, n_hidden) * 0.01
    b2 = cp.zeros(n_output)
    return W1, b1, W2, b2

def forward_propagation(x, W1, b1, W2, b2):
    z1 = cp.dot(x, W1.T) + b1
    a1 = relu(z1)
    z2 = cp.dot(a1, W2.T) + b2
    y_hat = softmax(z2)
    return z1, a1, z2, y_hat

def backward_propagation(x, y, W1, b1, W2, b2, a1, y_hat, reg_lambda):
    m = x.shape[0]
    delta_2 = y_hat - y
    dW2 = cp.dot(delta_2.T, a1) / m + reg_lambda * W2
    db2 = cp.sum(delta_2, axis=0) / m
    delta_1 = cp.dot(delta_2, W2) * relu_derivative(a1)
    dW1 = cp.dot(delta_1.T, x) / m + reg_lambda * W1
    db1 = cp.sum(delta_1, axis=0) / m
    return dW1, db1, dW2, db2

def load_mnist():
    from tensorflow.keras.datasets import mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = cp.asarray(x_train.reshape(-1, 784) / 255.0)
    x_test = cp.asarray(x_test.reshape(-1, 784) / 255.0)
    y_train = cp.eye(10)[y_train]
    y_test = cp.eye(10)[y_test]
    return x_train, y_train, x_test, y_test

def train(x_train, y_train, W1, b1, W2, b2, eta, epochs, reg_lambda):
    for epoch in range(epochs + 1):
        z1, a1, z2, y_hat = forward_propagation(x_train, W1, b1, W2, b2)
        loss = cross_entropy_loss_with_regularization(y_hat, y_train, W1, W2, reg_lambda)
        dW1, db1, dW2, db2 = backward_propagation(x_train, y_train, W1, b1, W2, b2, a1, y_hat, reg_lambda)
        W1 -= eta * dW1
        b1 -= eta * db1
        W2 -= eta * dW2
        b2 -= eta * db2
        if epoch % 100 == 0:
            print(f"Epoch {epoch + 1}, Loss: {loss:.4f}")
    return W1, b1, W2, b2

if __name__ == "__main__":
    x_train, y_train, x_test, y_test = load_mnist()
    n_input, n_hidden, n_output = 784, 128, 10
    eta = 0.01
    epochs = 1000
    lambdas = [0.01, 0.05, 0.1, 0.5, 1]
    best_accuracy = 0
    best_params = None

    for reg_lambda in lambdas:
        W1, b1, W2, b2 = initialize_parameters(n_input, n_hidden, n_output)
        W1, b1, W2, b2 = train(x_train, y_train, W1, b1, W2, b2, eta, epochs, reg_lambda)
        predictions = predict(x_test, W1, b1, W2, b2)
        test_labels = cp.argmax(y_test, axis=1)
        accuracy = cp.mean(predictions == test_labels)
        print(f"Test Accuracy: {accuracy * 100:.2f}% with lambda = {reg_lambda}")

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = (W1, b1, W2, b2)

    print(f"Best Test Accuracy: {best_accuracy * 100:.2f}%")

Epoch 1, Loss: 2.3536
Epoch 101, Loss: 2.3393
Epoch 201, Loss: 2.3132
Epoch 301, Loss: 2.2594


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
x_train, y_train, x_test, y_test = load_mnist()
# print(biggest_accuracy)
fig, axes = plt.subplots(1, 10, figsize=(20, 2))
for i, ax in enumerate(axes):
    ax.imshow(x_train[i].reshape(28, 28).get(), cmap='gray')
    ax.axis('off')
    ax.set_title(f'Label: {np.argmax(y_train[i])}')

plt.show()


In [ ]:
import matplotlib.pyplot as plt

x_train, y_train, x_test, y_test = load_mnist()

fig, axes = plt.subplots(1, 8, figsize=(16, 2))
for i, ax in enumerate(axes):
    ax.imshow(x_test[i].reshape(28, 28).get(), cmap='gray')
    ax.axis('off')
    ax.set_title(f'Label: {np.argmax(y_test[i])}')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import cupy as cp

W1, b1, W2, b2 = best_params
test_predictions = predict(x_test, W1, b1, W2, b2)
test_labels = cp.argmax(y_test, axis=1)  
accuracy = cp.mean(test_predictions == test_labels)  
print(f"Accuracy: {100 * accuracy.get():.2f}%")  

fig, axes = plt.subplots(1, 8, figsize=(16, 2))
for i, ax in enumerate(axes):
   
    ax.imshow(cp.asnumpy(x_test[i]).reshape(28, 28), cmap='gray')
    ax.axis('off')
   
    ax.set_title(f'Pred: {int(test_predictions[i].get())}\nTrue: {int(test_labels[i].get())}')

plt.show()
